In [1]:
import urllib.request 
from urllib import parse 
import json 
import re
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
from geopy.distance import geodesic

mykey=u'51cffc64fd38c249e18e2a8265ad06f9'

In [2]:
data=pd.read_excel('data/收费站info.xlsx')
data.columns=['idx','id','name','lon','lat']

In [3]:
mapping={}
for i in range(0,457):
    mapping[str(data['id'][i])]=i
reverse=dict(map(reversed, mapping.items()))

In [4]:
tollList=data['id'].tolist()
tollList=[str(i) for i in tollList]

In [5]:
conn=pd.DataFrame([])
FilePath='data/conn/data'
for root, dirs, files in os.walk(FilePath):
    for file in files:
        df=pd.read_csv(os.path.join(root, file),encoding='utf-8')
        conn=conn.append(df)
conn=conn.dropna()
conn.columns=['date','start','end','path','index']
conn.head()

,date,start,end,path,index
0,20200901,1623,1629,"1623,1633,1625,1627,1629",200000
1,20200901,3451,3459,"3451,3453,3455,3457,3459",200001
2,20200901,1833,1629,"1833,1831,1829,1801,1231,1229,1205,1201,1631,1629",200002
3,20200901,4013,4015,"4013,4015",200003
4,20200901,1011,1014,"1011,1014",200004


In [ ]:
# 构建连通性矩阵A
A=np.zeros((457,457))

In [48]:
directdist=np.ones((457,457))*np.inf
row, col = np.diag_indices_from(directdist)
directdist[row,col] = 0
directdist

array([[ 0., inf, inf, ..., inf, inf, inf],
       [inf,  0., inf, ..., inf, inf, inf],
       [inf, inf,  0., ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ...,  0., inf, inf],
       [inf, inf, inf, ..., inf,  0., inf],
       [inf, inf, inf, ..., inf, inf,  0.]])

In [49]:
for i in range(457):
    for j in range(457):
        if A[i,j]>0:
            go=data[data['id']==int(reverse[i])]
            to=data[data['id']==int(reverse[j])]
            directdist[i,j]=geodesic((float(go['lat']),float(go['lon'])), (float(to['lat']),float(to['lon']))).km

In [41]:
# (directdist==directdist.T).all()

True

In [14]:
rec=0

for record in conn.iterrows():
    RecFlag=False
    paths = record[1]['path'].split(',')
    for i in range(0,len(paths)):
        if(not paths[i] in tollList):
            continue
        else:
            if(RecFlag):
                A[mapping[rec],mapping[paths[i]]]=1
                A[mapping[paths[i]],mapping[rec]]=1
                rec=paths[i]
            else:
                RecFlag = True
                rec=paths[i]

In [50]:
np.save('data/conn/A',A)
np.save('data/conn/dist',directdist)

In [8]:
A=np.load('data/conn/A.npy')
directdist=np.load('data/conn/dist.npy')

In [3]:
Url=('http://restapi.amap.com/v3/geocode/regeo?key={key}&location={location}&radius=1000&extensions=all&roadlevel=0')
geo=[]
for record in data.iterrows():
    myloc=str(record[1]['lon'])+','+str(record[1]['lat'])
    myUrl = parse.quote(Url.format(key=mykey,location=myloc),safe="/:=&?#+!$,;'@()*[]")
    response = urllib.request.urlopen(myUrl)
    data_ = response.read()
    jsonData=json.loads(data_)
    break
#     format_=jsonData['regeocode']['addressComponent']['province'],jsonData['regeocode']['addressComponent']['city'],jsonData['regeocode']['addressComponent']['district']
#     geo.append(format_)

# location=pd.DataFrame(geo)
# location.columns=['prov','city','district']
# location

# df = pd.merge(data,location,how='inner',left_index=True,right_index=True)

In [10]:
# 验证最短路径假设
def Floyd(m):
    ret=m.copy()
    for k in range(len(ret)):        
        for i in range(len(ret)):     
            for j in range(len(ret)): 
                if ret[i][j] > ret[i][k] + ret[k][j]:
                    ret[i][j] = ret[i][k] + ret[k][j]
    return ret

In [11]:
A=np.load('data/conn/A.npy') #连通矩阵
# dist=A.copy()
# np.place(dist,dist==0,np.inf)
# row, col = np.diag_indices_from(dist)
# dist[row,col] = 0
dist=np.load('data/conn/dist.npy')
dists=Floyd(dist)

In [12]:
np.save('data/conn/dist_direct',dists)

In [54]:
result=[]

for path in conn['path']:
    flag=True
    for i in path.split(','):
        if i not in mapping.keys():
            flag=False
    if not flag:
        continue
    total=0
    for i in zip(path.split(',')[:-1],path.split(',')[1:]):
        total+=dist[mapping[i[0]],[mapping[i[1]]]]
    result.append({'Acutal':float(total),'Shortest':float(dists[mapping[path.split(',')[0]],mapping[path.split(',')[1]]]),'start':path.split(',')[0],'end':path.split(',')[-1]})

result=pd.DataFrame(result)
    

In [59]:
result['Equal']=result['Acutal']==result['Shortest']

In [61]:
result['Equal'].describe()
#最短距离假设误差大！

count     248637
unique         2
top        False
freq      138899
Name: Equal, dtype: object

In [62]:
result

,Acutal,Shortest,start,end,Equal
0,32.749057,14.264354,3451,3459,False
1,6.019255,6.019255,4013,4015,True
2,0.463268,0.463268,1011,1014,True
3,42.585615,16.712744,2027,2019,False
4,23.314859,23.314859,1223,1225,True
...,...,...,...,...,...
248632,15.147661,15.147661,1119,1121,True
248633,15.147661,15.147661,1119,1121,True
248634,25.329693,9.454249,1255,1537,False
248635,8.324320,8.324320,4315,4317,True
